# Preprocessing

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
import time

import torch
from torch.utils.data import DataLoader
import torch.utils.data as data_utils


In [ ]:
import torch



# Device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Path of the model (saved/to save)
modelFolder = './models/'

# When True, retrain the whole model
retrain = True

# Downsample the dataset
ds = True

# Size of the split
trainSize = 0.75
valSize = 0.05
testSize = 0.20

# Specify number of seconds for the window. Default: 16
window_size = 16

# Model hyper-parameters
batch_size = 4
learning_rate = 1e-3

# Seed for reproducibility
seed = 42

# Classes to drop in the dataset
classes_to_drop=[
    'stabf','stab']



In [ ]:
import numpy as np
import os
import pandas as pd
import random

from imblearn.under_sampling import RandomUnderSampler
from sklearn import preprocessing
from sklearn.metrics import f1_score
from torch.utils.data import Dataset

import torch
import torch.nn as nn



def setSeed(seed=seed):
    """
    Setting the seed for reproducibility
    """
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

setSeed()

def min_max_norm(self,col):
    self._data[col]=(self._data[col]-self._data[col].min())/(self._data[col].max()-self._data[col].min())


def std_scaler(self,col):
    self._data[col]=(self._data[col]-self._data[col].mean())/(self._data[col].std())


def f1(test_loader, model):
    f1 = 0
    with torch.no_grad():
        for i, (data, labels) in enumerate(test_loader):
            outputs = model(data)
            pred = outputs.data.max(1, keepdim=True)[1]
            f1 += f1_score(labels, pred, average='macro')
    avg_f1 = f1/len(test_loader)
    return (avg_f1)


class CustomDataset(Dataset):
    def __init__(self, file_path='/content/new_dataset.csv', classes_to_drop=classes_to_drop, window_size=window_size, normalize=True, normalize_method='mean_std', auth=False, target=None):

        self._window_size=window_size
        self._data=pd.read_csv(file_path)

        # if auth==True:
        #     if target != 'J':
        #         self._data = self._data[self._data['stabf'].isin([target, 'J'])]
        #     else:
        #         self._data = self._data[self._data['stabf'].isin([target, 'I'])]

        #     self._data['stabf'] = self._data['stabf'].apply(lambda x: target if x == target else 'Z')
        #     self._data['stabf'] = self._data['stabf'].map({target: 1, 'Z': 0}).fillna(0).astype(int)


        # # Random Undersampling
        # X = self._data.drop('stabf', axis=1)
        # y = self._data['stabf']

        # # sampler = RandomUnderSampler(sampling_strategy='not minority', random_state=seed)
        # # X_resampled, y_resampled = sampler.fit_resample(X, y)

        # # X_resampled['Class'] = y_resampled
        # self._data = X

        # The data is sorted by Class A,B,C the indexes of the dataframe have restarted by ignore index
        self._data = self._data.sort_values(by=['stabf'], inplace=False,ignore_index = True)

        # class_uniq contains the letters of the drivers A,B and it loops across all of them
        for class_uniq in list(self._data['stabf'].unique()):
            # Find the total number of elements belonging to a class
            tot_number=sum(self._data['stabf']==class_uniq)
            # Number of elements to drop so that the class element is divisible by window size
            to_drop=tot_number%window_size
            # Returns the index of the first element of the class
            index_to_start_removing=self._data[self._data['stabf']==class_uniq].index[0]
            # Drop element from first element to the element required
            self._data.drop(self._data.index[index_to_start_removing:index_to_start_removing+to_drop],inplace=True)


        # Resetting index of dataframe after dropping values
        self._data = self._data.reset_index()
        self._data = self._data.drop(['index'], axis=1)

        index_starting_class=[] # This array contains the starting index of each class in the df
        for class_uniq in list(self._data['stabf'].unique()):
            # Appending the index of first element of each clas
            index_starting_class.append(self._data[self._data['stabf']==class_uniq].index[0])

        # Create the sequence of indexs of the windows
        sequences=[]
        for i in range(len(index_starting_class)):
            # Check if beginning of next class is there
            if i!=len(index_starting_class)-1:
                ranges=np.arange(index_starting_class[i], index_starting_class[i+1])
            else:
                ranges = np.arange(index_starting_class[i], len(self._data))
            for j in range(0,len(ranges),int(self._window_size/2)):
                if len(ranges[j:j+self._window_size])==16:
                    sequences.append(ranges[j:j+self._window_size])
        self._sequences=sequences


        # Take only the 'Class' which are the actual labels and store it in the labels of self
        self._labels=self._data['stabf']
        # Dropping columns which have constant measurements because they would return nan in std
        self._data.drop(classes_to_drop, inplace=True, axis=1)

        # Function to normalize the data either with min_max or mean_std
        if normalize and not auth:
            for col in self._data.columns:
                if normalize_method=='min_max':
                    min_max_norm(self,col)
                elif normalize_method=="mean_std":
                    std_scaler(self,col)

        # Create the array holding the windowed multidimensional arrays
        X=np.empty((len(sequences), self._window_size, len(self._data.columns)))
        y=[]

        for n_row, sequence in enumerate(sequences):
            X[n_row,:,:]=self._data.iloc[sequence]
            # The corresponding driver of the sequence is the driver at first sequence
            y.append(self._labels[sequence[0]])

        assert len(y)==len(X)
        # Assign the windowed dataset to the X of self
        self._X= X

        # Targets is a transformed version of y with drivers are encoded into 0 to 9
        targets = preprocessing.LabelEncoder().fit_transform(y)
        class_labels = encoder.classes_
        for code, label in enumerate(class_labels):
          print(f'Code: {code} -> Label: {label}')
        targets = torch.as_tensor(targets)  # Just converting it to a pytorch tensor
        self._y=targets # Assign it to y of self


    def __len__(self):
        return len(self._X)


    def __getitem__(self, index):
        return torch.FloatTensor(self._X[index,:,:]), self._y[index]


def evaluate(model, dataloader, criterion):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    y_true = []
    y_pred = []

    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        inputs = inputs
        labels = labels

        # Forward pass
        with torch.no_grad():
            outputs = model(inputs)
            loss = criterion(outputs, labels)

        _, preds = torch.max(outputs, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

        # Collect predictions and true labels
        y_true += labels.data.cpu().numpy().tolist()
        y_pred += preds.cpu().numpy().tolist()

    # Calculate accuracy and loss
    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = running_corrects.double() / len(dataloader.dataset)
    epoch_f1 = f1_score(y_true, y_pred, average='macro')

    return epoch_loss, epoch_acc, epoch_f1


def evaluateBinary(model, dataloader, criterion):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    y_true = []
    y_pred = []

    for inputs, labels in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Forward pass
        with torch.no_grad():
            outputs = model(inputs)
            # loss = criterion(outputs, labels)
            loss = criterion(outputs.squeeze(), labels.float())

        _, preds = torch.max(outputs, 1)
        # preds = (outputs > 0.5).float()
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

        # Collect predictions and true labels
        y_true += labels.data.cpu().numpy().tolist()
        y_pred += preds.cpu().numpy().tolist()

    # Calculate accuracy and loss
    epoch_loss = running_loss / len(dataloader.dataset)
    epoch_acc = running_corrects.double() / len(dataloader.dataset)
    epoch_f1 = f1_score(y_true, y_pred, average='macro')

    return epoch_loss, epoch_acc, epoch_f1



In [ ]:
dataset_path = '/content/smart_grid_stability_augmented.csv'
df = pd.read_csv(dataset_path)
df

In [ ]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
df['stabf'] = encoder.fit_transform(df['stabf'])

# Retrieve the mapping of numerical codes to original class labels
class_labels = encoder.classes_

# Display the mapping
for code, label in enumerate(class_labels):
    print(f'Code: {code} -> Label: {label}')
df

In [ ]:
df.to_csv('new_dataset.csv', index=False)
df

In [ ]:
import torch
import numpy as np
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True


In [ ]:
a = CustomDataset()

# Defining sizes
train_size = int(trainSize * len(a))
val_size = int(valSize * len(a))
test_size = len(a)-train_size-val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
    a, [train_size, val_size, test_size])


train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=False,
                                           drop_last=True)

validation_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                                batch_size=batch_size,
                                                shuffle=False,
                                                drop_last=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False,
                                          drop_last=True)

# Reproduced model from litreature

In [ ]:
import torch.optim as optim


class MLP_reproduced(torch.nn.Module):
    def __init__(self, batch_size, window_size, num_features, dropout_rate=0.25):
        super(MLP_reproduced, self).__init__()
        self.nn1 = torch.nn.Linear(num_features, 288)
        self.relu1 = torch.nn.ReLU()
        self.dropout = torch.nn.Dropout(p=dropout_rate)
        self.nn2 = torch.nn.Linear(288, 24)
        self.relu2 = torch.nn.ReLU()
        self.nn3 = torch.nn.Linear(24, 12)
        self.relu3 = torch.nn.ReLU()
        self.fc = torch.nn.Linear(12, 1)  # Output size is 1 for binary classification
        self.sigmoid = torch.nn.Sigmoid()  # Sigmoid activation for binary classification

    def forward(self, x):
        nn1_out = self.relu1(self.nn1(x))
        nn1_out = self.dropout(nn1_out)
        nn2_out = self.relu2(self.nn2(nn1_out))
        nn3_out = self.relu3(self.nn3(nn2_out))
        fc_out = self.fc(nn3_out)
        out = self.sigmoid(fc_out)
        return out[:, -1, :]

model = MLP_reproduced(batch_size, window_size, 12).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

retrain = False

if not os.path.exists('./models/rnn_auth.pt') or retrain:
    # Training loop
    for epoch in range(50):
        model.train()
        total_loss = 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels.float())
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        average_loss = total_loss / len(train_loader)
        print(f'[💪 EPOCH {epoch+1}/50] Loss: {average_loss:.3f}')


In [ ]:
# Calculate accuracy
correct_predictions = 0
total_samples = 0

y_true = []
y_pred = []

for inputs, labels in test_loader:
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = model(inputs)
    predictions = (outputs > 0.5).float()

    for p, l in zip(predictions, labels.float()):
        if p == l:
            correct_predictions += 1

    total_samples += labels.size(0)

    y_true.extend(labels.cpu().numpy())
    y_pred.extend(predictions.cpu().numpy())

acc = correct_predictions/total_samples
f1 = f1_score(y_true, y_pred, average='binary')

print('[👑 TEST GRU AUTH]\n')
print(f'[🎯 ACCURACY] {acc:.3f}')
print(f'[⚖️ F1 SCORE] {f1:.3f}')

# GAN-GRID Attack

In [ ]:
class Generator(nn.Module):
    def __init__(self, batch_size, window_size, num_features,):
        super(Generator, self).__init__()
        self.batch_size = batch_size
        self.num_features = num_features
        self.window_size = window_size
        self.layer1 = nn.Linear(num_features, 128)
        self.layer2 = nn.Linear(128, 256)
        self.layer3 = nn.Linear(256, 512)
        self.layer4 = nn.Linear(512, batch_size*window_size)
        self.layer5 = nn.Linear(batch_size*window_size, num_features)

        self.leaky_relu = nn.LeakyReLU(0.2)

    def forward(self, x):
        x = self.leaky_relu(self.layer1(x))
        x = self.leaky_relu(self.layer2(x))
        x = self.leaky_relu(self.layer3(x))
        x = self.leaky_relu(self.layer4(x))
        x = self.layer5(x)
        return x

In [ ]:
def train_gan(generator, surrogate, label, train_loader, num_epochs=100, lr=0.001, device=torch.device('cpu'), ml=False, num_episodes=150):

    losses = []

    if not ml:
        generator = generator.to(device)
        surrogate = surrogate.to(device)

        # for model in surrogate.models:
        #     model.to(device)
        #     model.train()

    # Define the loss function and optimizer
    binary_cross_entropy_loss = nn.BCEWithLogitsLoss()
    generator_optimizer = torch.optim.Adam(generator.parameters(), lr=lr)

    # Define the reinforcement learning parameters
    max_episode_length = 10
    alpha = 0.1
    gamma = 0.9

    for episode in range(num_episodes):
        # Initialize the latent input and the episode reward
        latent_input = torch.randn(4, 16, 12).to(device)
        episode_reward = 0

        for step in range(max_episode_length):
            # Generate a sample with the current latent input
            fake_input = generator(latent_input)

            # Evaluate the sample with the surrogate model
            if not ml:
                surrogate_output = surrogate(fake_input)
            else:
                surrogate_output = []
                # Looping through each
                for group in fake_input:
                    # Flatten the group to make it
                    flat_group = group.view(-1, group.size(-1)).detach().numpy()
                    # Get the probabilities
                    probabilities = surrogate.predict_proba(flat_group)
                    mean_probabilities = np.mean(probabilities, axis=0)
                    # Append the probabilities to the array
                    surrogate_output.append(mean_probabilities)
                with torch.no_grad():
                    surrogate_output = torch.tensor(surrogate_output, requires_grad=True)

            predictions = (surrogate_output > 0.5).float()
            targets = torch.randint_like(predictions, 0, 2)
            reward = (predictions == targets).float().mean().item()
            episode_reward += reward

            # Update the latent input using reinforcement learning
            td_error = reward - episode_reward
            latent_input += alpha * td_error * gamma**step * torch.randn_like(latent_input)

        # Update the generator using the final latent input of the episode
        generator_optimizer.zero_grad()
        fake_input = generator(latent_input)

        if not ml:
            surrogate_output = surrogate(fake_input)
        else:
            surrogate_output = []
            # Looping through
            for group in fake_input:
                flat_group = group.view(-1, group.size(-1)).detach().numpy()
                # Get the probabilities
                probabilities = surrogate.predict_proba(flat_group)
                mean_probabilities = np.mean(probabilities, axis=0)
                # Append the probabilities to the array
                surrogate_output.append(mean_probabilities)
            with torch.no_grad():
                surrogate_output = torch.tensor(surrogate_output, requires_grad=True)

        target_labels = targets.view(-1, 1).float()
        target_labels = torch.full_like(target_labels, label)

        generator_loss = binary_cross_entropy_loss(surrogate_output, target_labels)

        if ml:
            generator_optimizer.zero_grad()

        generator_loss.backward()
        generator_optimizer.step()

        losses.append(generator_loss.item())

        if episode % 10 == 0:
            print(f'[⏭️ EP {episode}/{num_episodes} | D{label}] LOSS: {round(generator_loss.item(), 3)}')

    print()

    return generator, losses

In [ ]:
lr = 3e-3

generators = []
losses = []

inputs, classes = next(iter(train_loader))

# For each driver
for d in range(1):
        print(f'[🤖 GENERATORS] Label {d}')

        batch_size, window_size, num_features = inputs.shape
        generator = Generator(batch_size, window_size, num_features)
        surrogate_model = model

        generator, loss = train_gan(generator, surrogate_model, train_loader=train_loader, num_epochs=20, lr=lr, label=0, ml=False, num_episodes=100)
        print()

        generators.append(generator)
        losses.append(loss)

In [ ]:
results = []

threshold = 0.5

for i in range(1):
    predicted_labels = []
    generator = generators[i].to(device)

    for batch in test_loader:
        input_batch, true_labels = batch[0].to(device), batch[1].to(device)
        # Generate data
        generated_data = generator(torch.randn(4, 16, 12).to(device))
        # generated_data = generated_data * ones_tensor

        # Add the result to the ones tensor
        final_result =  generated_data

        # Get the surrogate outputs for each sample in the generated data
        surrogate_outputs = model(final_result)

        # Apply the threshold for binary classification
        predicted_labels_batch = (surrogate_outputs > threshold).float()

        # Append the predicted labels to the lists
        predicted_labels.extend(predicted_labels_batch.squeeze().tolist())  # Squeeze the tensor

    asr = predicted_labels.count(0) / len(predicted_labels)
    results.append(asr)
    print(f'[👑 DRIVER {i}] ASR: {round(asr, 3)}')